In [80]:
import numpy as np
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test


In [81]:
predictions = {}

In [82]:
stf_pred = {}
stf_pred[1] = np.load("../code/baseline/result/stf-pred-12.npy").item()[1]
stf_pred[3] = np.load("../code/baseline/result/stf-pred-34.npy").item()[3]
predictions['stf'] = stf_pred

In [83]:
mtf_pred = {}
mtf_pred[1] = np.load("../code/baseline/result/mtf-pred-12.npy").item()[1]
mtf_pred[3] = np.load("../code/baseline/result/mtf-pred-34.npy").item()[3]
predictions['mtf'] = mtf_pred

In [84]:
active_threshold = {'hvac':230, 'fridge':20, 'dr':250, 'dw':55, 'mw':10}

In [85]:
num_homes = {0:14, 1:14, 2:14, 3:13, 4:13}

In [86]:
# calculate gt
test_gt = {}
valid_gt = {}
for fold_num in range(5):
    test_gt[fold_num] = {}
    valid_gt[fold_num] = {}
    train, test = get_train_test(1, 5, fold_num)
    valid = train[int(0.8*len(train)):].copy()
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        test_gt[fold_num][appliance] = test[:, idx+1]
        valid_gt[fold_num][appliance] = valid[:, idx+1]

In [87]:
active = {}
for dataset in [1, 3]:
    active[dataset] = {}
    for fold_num in range(5):
        active[dataset][fold_num] = {}
        for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
            active[dataset][fold_num][appliance] = (test_gt[fold_num][appliance] > active_threshold[appliance])*1

In [88]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (stf_pred[dataset][fold_num][:, idx] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [89]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(appliance, "Precision: ", pr, "Recall: ", recall)


hvac Precision:  0.811093983185 Recall:  0.977586308995
fridge Precision:  0.978629785109 Recall:  0.920857835046
dr Precision:  0.125 Recall:  0.05203969129
dw Precision:  0.0887822435513 Recall:  0.0397689103856
mw Precision:  0.174036595509 Recall:  0.484632086852
hvac Precision:  0.796154888292 Recall:  0.956993545331
fridge Precision:  0.977819653745 Recall:  0.96098003122
dr Precision:  0.113482056256 Recall:  0.0386990077178
dw Precision:  0.0626020685901 Recall:  0.0463522773075
mw Precision:  0.15893247776 Recall:  0.262919179735


In [90]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (mtf_pred[dataset][fold_num][:, idx] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [91]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(appliance, "Precision: ", pr, "Recall: ", recall)


hvac Precision:  0.816622527261 Recall:  0.972323384852
fridge Precision:  0.978581370906 Recall:  0.96372161205
dr Precision:  0.125177003682 Recall:  0.0487320837927
dw Precision:  0.0813429472735 Recall:  0.0485019481392
mw Precision:  0.20485015241 Recall:  0.366417370326
hvac Precision:  0.795939687083 Recall:  0.975676555931
fridge Precision:  0.978608407768 Recall:  0.985699019185
dr Precision:  0.106988783434 Recall:  0.0410143329658
dw Precision:  0.0734101579172 Recall:  0.0693268843208
mw Precision:  0.212929915639 Recall:  0.379975874548


In [92]:
sc_pred = np.load("../code/baseline/result/sc-non-disc-pred.npy").item()
predictions['sc'] = sc_pred

In [93]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (sc_pred[dataset][fold_num][:, idx] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [94]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(appliance, "Precision: ", pr, "Recall: ", recall)


hvac Precision:  1.0 Recall:  0.924735003257
fridge Precision:  0.983653614 Recall:  0.615462515876
dr Precision:  0.179126572909 Recall:  0.0266813671444
dw Precision:  0.0703951506834 Recall:  0.16539029961
mw Precision:  0.1330680745 Recall:  0.414716525935
hvac Precision:  0.78580687565 Recall:  0.955786995914
fridge Precision:  0.9775432205 Recall:  0.97883947205
dr Precision:  0.127014140085 Recall:  0.170341786108
dw Precision:  0.0681247196052 Recall:  0.408034394733
mw Precision:  0.116767621932 Recall:  0.788757539204


In [95]:
dsc_pred = np.load("../code/baseline/result/DSC-pred.npy").item()
predictions['dsc'] = dsc_pred

In [96]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (dsc_pred[dataset][fold_num][:, idx] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [97]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(appliance, "Precision: ", pr, "Recall: ", recall)


hvac Precision:  0.760138337537 Recall:  0.979414638479
fridge Precision:  0.982917031262 Recall:  0.553071969294
dr Precision:  0.133450395083 Recall:  0.0167585446527
dw Precision:  0.0742976549803 Recall:  0.0429934166331
mw Precision:  0.159927536232 Recall:  0.212979493366
hvac Precision:  0.753863216266 Recall:  0.981139337952
fridge Precision:  0.981540236516 Recall:  0.456959978515
dr Precision:  0.114399544678 Recall:  0.0443219404631
dw Precision:  0.0607164541591 Recall:  0.174660755072
mw Precision:  0.161823263176 Recall:  0.315705669481


In [98]:
cnn_ind_params = {}
# cnn_ind_params[1] = np.load("../code/baseline/result/cnn-individual-new-param-1.npy").item()
# cnn_ind_params[3] = np.load("../code/baseline/result/cnn-individual-new-param-3.npy").item()
cnn_ind_params[1] = np.load("../code/baseline/result/cnn-individual-param-1.npy").item()
cnn_ind_params[3] = np.load("../code/baseline/result/cnn-individual-param-3.npy").item()

In [99]:
gid = 0
cnn_ind_pred = {}
for dataset in [1, 3]:
    cnn_ind_pred[dataset] = {}
    for fold_num in range(5):
        cnn_ind_pred[dataset][fold_num] = {}
        iters = cnn_ind_params[dataset][fold_num]['iters']
        lr = cnn_ind_params[dataset][fold_num]['lr']
        for appliance in APPLIANCE_ORDER[1:-1]:
            directory = "../code/baseline/cnn-tree/{}/{}/{}/200000/0.0/".format(dataset, fold_num, lr)
            filename = "test-pred-[\'{}\'].npy".format(appliance)
            try:
                cnn_ind_pred[dataset][fold_num][appliance] = np.load(directory+filename).item()[iters][0].reshape(-1, 112, 24)
            except:
                print("CUDA_VISIBLE_DEVICES={} python cnn-tree-save-test.py {} {} 200000 0 {} {} &".format(gid, dataset, lr, fold_num, appliance))
                gid += 1
                if gid == 4:
                    gid = 0


In [100]:
for dataset in [1,3]:
    for idx, appliance in enumerate(order):
        error = 0
        for fold_num in range(5):
        
            abs_error = np.abs(cnn_ind_pred[dataset][fold_num][appliance] - test_gt[fold_num][appliance])
            error += abs_error.mean()*num_homes[fold_num]
        print(appliance, error/68)

dr 58.2060223639
hvac 171.678719181
mw 8.56300932797
dw 15.4727047643
fridge 31.5697526663
dr 68.3298168953
hvac 321.134156619
mw 8.58014962559
dw 15.6936825714
fridge 31.9884790963


In [101]:
predictions['ind cnn'] = cnn_ind_pred

In [102]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (cnn_ind_pred[dataset][fold_num][appliance] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [103]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(recall)


0.964825013324
0.998086488483
0.360749724366
0.0
0.00303980699638
0.95124948185
0.996167381902
0.0471885336273
0.0
0.0143305186972


/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in long_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [104]:
cnn_tree_params = {}
cnn_tree_params[1] = np.load("../code/baseline/result/cnn-tree-new-param-1.npy").item()
cnn_tree_params[3] = np.load("../code/baseline/result/cnn-tree-final-param-3.npy").item()

In [105]:
# get prediction for cnn tree
cnn_tree_pred = {}
for dataset in [1, 3]:
    cnn_tree_pred[dataset] = {}
    for fold_num in range(5):
        cnn_tree_pred[dataset][fold_num] = {}

        lr = cnn_tree_params[dataset][fold_num]['lr']
        iters = cnn_tree_params[dataset][fold_num]['iters']
        order = cnn_tree_params[dataset][fold_num]['order']

        o = "\', \'".join(str(x) for x in order)
        directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num,lr)
        filename = "test-pred-[\'{}\'].npy".format(o)

        full_path = directory + filename
        cnn_pred = np.asarray(np.load(full_path).item()[iters])

        for idx, appliance in enumerate(order):
            cnn_tree_pred[dataset][fold_num][appliance] = cnn_pred[idx].reshape(-1, 112,24)
    

In [106]:
predictions['tree cnn'] = cnn_tree_pred

In [107]:
for dataset in [1,3]:
    for idx, appliance in enumerate(order):
        error = 0
        for fold_num in range(5):
        
            abs_error = np.abs(cnn_tree_pred[dataset][fold_num][appliance] - test_gt[fold_num][appliance])
            error += abs_error.mean()*num_homes[fold_num]
        print(error/68)

87.5339624734
163.315904141
8.63536474212
16.4218572378
37.7565564686
69.0058276011
306.384807266
9.00085921228
15.4479628075
33.7374402039


In [108]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (cnn_tree_pred[dataset][fold_num][appliance] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [109]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(recall)


0.969777047433
0.969730709622
0.492613009923
0.000537417707913
0.0135102533172
0.959036536981
0.991607405625
0.148732083793
0.0
0.0248974668275


/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in long_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [110]:
num_folds=5
def mean_predictions(dataset):
    out = []
    for cur_fold in range(5):
        train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
        pred = np.zeros_like(test[:, 1:, :, :])
#         print(pred.shape)
        
        for appliance in range(test.shape[1]-1):
            for day in range(test.shape[2]):
                pred[:, appliance, day, :] = train[:, appliance+1, day, :].mean(axis=0)
#         print(pred[0, 1, 0, 20])
        out.append(pred)
#     return np.concatenate(out)
    return out


In [111]:
tensor = np.load('../2015-5appliances-true-agg.npy')
# tensor = np.load('../2015-5appliances.numpy.npy')
from sklearn.metrics import mean_absolute_error

dataset = 1
mean_pred = {}
for dataset in [1, 3]:

    gt = tensor[:, 1:, :, :]
    mean_pred[dataset] = mean_predictions(dataset)
# Clamping prediction to aggregate
predictions['mean'] = mean_pred

In [112]:
TP = {}
TP_FP = {}
TP_FN = {}
for dataset in [1, 3]:
    TP[dataset] = {}
    TP_FP[dataset] = {}
    TP_FN[dataset] = {}
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        TP[dataset][appliance] = 0
        TP_FP[dataset][appliance] = 0
        TP_FN[dataset][appliance] = 0
        for fold_num in range(5):
            test_active = (mean_pred[dataset][fold_num][:, idx] > active_threshold[appliance])*1
            TP[dataset][appliance] += np.multiply(test_active, active[dataset][fold_num][appliance]).sum()
            TP_FP[dataset][appliance] += test_active.sum()
            TP_FN[dataset][appliance] += active[dataset][fold_num][appliance].sum()

In [113]:
for dataset in [1, 3]:
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        pr = TP[dataset][appliance]/TP_FP[dataset][appliance]
        recall = TP[dataset][appliance]/TP_FN[dataset][appliance]
        
        print(appliance, "Precision: ", pr, "Recall: ", recall)


hvac Precision:  0.753081105573 Recall:  0.984218629715
fridge Precision:  0.977815344888 Recall:  1.0
dr Precision:  0.10296474359 Recall:  0.0283351708931
dw Precision:  0.0756002595717 Recall:  0.031304581486
mw Precision:  0.170584519218 Recall:  0.495102533172
hvac Precision:  0.753081105573 Recall:  0.984218629715
fridge Precision:  0.977815344888 Recall:  1.0
dr Precision:  0.10296474359 Recall:  0.0283351708931
dw Precision:  0.0756002595717 Recall:  0.031304581486
mw Precision:  0.170584519218 Recall:  0.495102533172


In [114]:
# error of active parts

In [115]:
error = {}
for dataset in [1, 3]:
    error[dataset] = {}
    for method in ['sc']:
        error[dataset][method] = {}
        error[dataset][method]['Active Error'] = {}
        error[dataset][method]['Inactive Error'] = {} 
        print("Dataset ", dataset)
        for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
            active_error = []
            inactive_error = []
            print(appliance)
            for fold_num in range(5):
                abs_error = np.abs(predictions[method][dataset][fold_num][:, idx]-test_gt[fold_num][appliance])
                active_error.append(abs_error[active[dataset][fold_num][appliance] == 1])
                inactive_error.append(abs_error[active[dataset][fold_num][appliance] == 0])
    #             print(appliance, fold_num, abs_error[active[dataset][fold_num][appliance] == 1].mean())
            print("active error", np.hstack(active_error).mean())
            print("inactive error", np.hstack(inactive_error).mean())


Dataset  1
hvac
active error 276.623251431
inactive error 4.35813867811
fridge
active error 118.957349014
inactive error 79.8034176483
dr
active error 1259.90720008
inactive error 19.2902113243
dw
active error 341.543323579
inactive error 17.7608014526
mw
active error 62.6150733512
inactive error 21.8516790999
Dataset  3
hvac
active error 512.570701381
inactive error 499.921435962
fridge
active error 443.538620837
inactive error 372.362016012
dr
active error 1179.77652982
inactive error 55.7073357965
dw
active error 310.611729639
inactive error 53.8440615338
mw
active error 131.185307906
inactive error 96.0238578033


In [116]:
for dataset in [1, 3]:
    for method in ['ind cnn']:
#         error[dataset][method] = {}
        print("Dataset ", dataset)
        for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
            active_error = []
            inactive_error = []
            print(appliance)
            for fold_num in range(5):
                abs_error = np.abs(predictions[method][dataset][fold_num][appliance]-test_gt[fold_num][appliance])
                active_error.append(abs_error[active[dataset][fold_num][appliance] == 1])
                inactive_error.append(abs_error[active[dataset][fold_num][appliance] == 0])
    #             print(appliance, fold_num, abs_error[active[dataset][fold_num][appliance] == 1].mean())
            print("active error", np.hstack(active_error).mean())
            print("inactive error", np.hstack(inactive_error).mean())


Dataset  1
hvac
active error 190.418369125
inactive error 118.590903615
fridge
active error 30.6486913356
inactive error 72.1666381348
dr
active error 905.810084417
inactive error 13.9507013028
dw
active error 369.498791383
inactive error 0.444752587056
mw
active error 56.1798562447
inactive error 2.47350394814
Dataset  3
hvac
active error 350.244203657
inactive error 238.667898555
fridge
active error 31.0780380464
inactive error 72.1172629239
dr
active error 1262.3390757
inactive error 5.98789870036
dw
active error 369.335260598
inactive error 0.682052289559
mw
active error 56.0041332298
inactive error 2.51530867139


In [139]:
dataset = 1

ind_cnn = np.ones((68, 5, 112, 24))
for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
    app = predictions['ind cnn'][dataset][0][appliance]
    for fold_num in range(1, 5):
        app = np.vstack((app, predictions['ind cnn'][dataset][fold_num][appliance]))
#         print(predictions['ind cnn'][1][fold_num][appliance].shape)
    ind_cnn[:, idx] = app

In [140]:
tree_cnn = np.ones((68, 5, 112, 24))
for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
    app = predictions['tree cnn'][dataset][0][appliance]
    for fold_num in range(1, 5):
        app = np.vstack((app, predictions['tree cnn'][dataset][fold_num][appliance]))
#         print(predictions['ind cnn'][1][fold_num][appliance].shape)
    tree_cnn[:, idx] = app

In [146]:
cnn_tree_params = {}
cnn_tree_params[3] = np.load("../code/baseline/result/cnn-tree-final-residual-param-3.npy").item()

In [147]:
# get prediction for cnn tree
cnn_tree_pred = {}
for dataset in [3]:
    cnn_tree_pred[dataset] = {}
    for fold_num in range(5):
        cnn_tree_pred[dataset][fold_num] = {}

        lr = cnn_tree_params[dataset][fold_num]['lr']
        iters = cnn_tree_params[dataset][fold_num]['iters']
        order = cnn_tree_params[dataset][fold_num]['order']

        o = "\', \'".join(str(x) for x in order)
        directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num,lr)
        filename = "test-pred-[\'{}\'].npy".format(o)

        full_path = directory + filename
        cnn_pred = np.asarray(np.load(full_path).item()[iters])

        for idx, appliance in enumerate(order):
            cnn_tree_pred[dataset][fold_num][appliance] = cnn_pred[idx].reshape(-1, 112,24)
    

In [149]:
tree_cnn_r = np.ones((68, 5, 112, 24))
for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
    app = cnn_tree_pred[3][0][appliance]
    for fold_num in range(1, 5):
        app = np.vstack((app, cnn_tree_pred[3][fold_num][appliance]))
#         print(predictions['ind cnn'][1][fold_num][appliance].shape)
    tree_cnn_r[:, idx] = app

In [165]:
from scipy import stats

stats.ttest_ind(tree_cnn_r[4].reshape(-1, 1), tree_cnn[4].reshape(-1, 1))

Ttest_indResult(statistic=array([-3.2802119]), pvalue=array([ 0.00103861]))

In [153]:
tensor = np.load("../2015-5appliances.numpy.npy")

In [160]:
mask = np.zeros((68, 5, 112, 24))
for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
    k = (tensor[:, idx+1] > active_threshold[appliance])*1
    mask[:, idx] = k

In [159]:
mask = mask.reshape(-1, 1)

In [ ]:
[active[dataset][fold_num][appliance] == 1]